In [1]:
%mkdir -p /content/drive/MyDrive/GSE182308
%cd /content/drive/MyDrive/GSE182308

/content/drive/MyDrive/GSE182308


In [2]:
!wget "https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE182308&format=file" -O GSE182308_RAW.tar
!tar -xvf GSE182308_RAW.tar

--2024-08-08 13:55:56--  https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE182308&format=file
Resolving www.ncbi.nlm.nih.gov (www.ncbi.nlm.nih.gov)... 130.14.29.110, 2607:f220:41e:4290::110
Connecting to www.ncbi.nlm.nih.gov (www.ncbi.nlm.nih.gov)|130.14.29.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 508354560 (485M) [application/x-tar]
Saving to: ‘GSE182308_RAW.tar’

GSE182308_RAW.tar   100%[===================>] 484.80M  4.71MB/s    in 55s     

2024-08-08 13:56:52 (8.89 MB/s) - ‘GSE182308_RAW.tar’ saved [508354560/508354560]

GSM5526487_10xRNA_Veh_GEMwell1_filtered_feature_bc_matrix.h5
GSM5526487_10xRNA_Veh_GEMwell1_sgRNAcounts.csv.gz
GSM5526488_10xRNA_Veh_GEMwell2_filtered_feature_bc_matrix.h5
GSM5526488_10xRNA_Veh_GEMwell2_sgRNAcounts.csv.gz
GSM5526489_10xRNA_ITC_GEMwell3_filtered_feature_bc_matrix.h5
GSM5526489_10xRNA_ITC_GEMwell3_sgRNAcounts.csv.gz
GSM5526490_10xRNA_ITC_GEMwell4_filtered_feature_bc_matrix.h5
GSM5526490_10xRNA_ITC_GEMwell4_sgRNAc

In [6]:
!zcat GSM5526487_10xRNA_Veh_GEMwell1_sgRNAcounts.csv.gz | head

Output hidden; open in https://colab.research.google.com to view.

In [2]:
%pip install scanpy python-dotenv -q

import os
import tarfile
import json
import gzip

import scanpy as sc
import anndata as ad
import pandas as pd

from dotenv import load_dotenv
load_dotenv(dotenv_path='/content/drive/MyDrive/.gse.env')
COLS = json.loads(os.environ.get('OBS_COLS'))
ORGANISMS = json.loads(os.environ.get('ORGANISMS'))

In [8]:
adatas = {}
for j, sample in enumerate([f'GEMwell{i}' for i in range(1, 5)]):
    file = f"GSM55264{87+j}_10xRNA_Veh_{sample}_filtered_feature_bc_matrix.h5" \
            if j < 2 else f"GSM55264{87+j}_10xRNA_ITC_{sample}_filtered_feature_bc_matrix.h5"
    adata = sc.read_10x_h5(file)
    adata.var_names_make_unique()
    obs = pd.read_csv(f"{sample}_sgrna_assignment.csv", index_col=1)
    obs.drop(columns=['Unnamed: 0'], inplace=True)
    adata.obs.index = adata.obs.index.str.split('-').str[0] + f'-{j+1}'
    adata = adata[obs.index]
    adata.obs = obs
    adata.obs.head()
    adatas[sample] = adata

adata = ad.concat(adatas, label='batch')
adata.write_h5ad('GSE182308_merged.h5ad')

/usr/local/lib/python3.10/dist-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/usr/local/lib/python3.10/dist-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/usr/local/lib/python3.10/dist-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/usr/local/lib/python3.10/dist-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/usr/local/lib/python3.10/dist-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_name

In [3]:
adata = sc.read_h5ad("./GSE182308_merged.h5ad")
adata.obs.head()

In [10]:
adata.obs['demuxType'].value_counts()

,count
demuxType,
doublet,58254
singlet,36706
unknown,6854


In [11]:
adata[adata.obs['demuxType'] == 'singlet'].obs['sgRNAtarget'].value_counts()

,count
sgRNAtarget,
REL,2176
HIF1A,2020
CHUK,1966
ARPC3,1956
CEBPD,1867
CEBPB,1755
MAP3K14,1702
JUN,1691
STAT2,1563


In [12]:
adata = adata[adata.obs['demuxType'] != 'unknown']
adata.obs['perturbation_name'] = adata.obs['demuxAssignment'].apply(
    lambda x: '+'.join([
        'non-targeting' if y == 'NTC' else y for y in x.split(',')
    ])
)
adata.obs['perturbation_name'] = adata.obs['perturbation_name'].apply(
    lambda x: '+'.join([
        y for y in x.split('+') if y != 'unknown'
    ])
)

adata.obs['condition'] = adata.obs['perturbation_name'].apply(
    lambda x: x if x == 'non-targeting' else 'targeting'
)
adata.obs['treatment'] = adata.obs['batch'].apply(
    lambda x: "IL1a+TNF+C1q" if x[-1] in ['1', '2'] else 'vehicle control'
)

<ipython-input-12-a455545d3c3c>:2: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['perturbation_name'] = adata.obs['demuxAssignment'].apply(


In [5]:
adata.obs['crispr_type'] = 'CRISPRi'
adata.obs['cancer_type'] = 'non-cancer'
adata.obs['cell_type'] = 'astrocytes'
adata.obs['organism'] = ORGANISMS[0]

In [6]:
missing = [col for col in COLS if col not in adata.obs]
print(f"Missing following columns: \n {missing}")

if 'organism' in adata.obs_keys():
    assert adata.obs['organism'].isin(ORGANISMS).all(), "Invalid organism naming"

Missing following columns: 
 []


In [14]:
adata.write_h5ad("./GSE182308_merged.h5ad")

In [13]:
adata.obs['perturbation_name'].value_counts()

,count
perturbation_name,
REL,2345
HIF1A,2143
CHUK,2089
ARPC3,2076
CEBPD,1974
...,...
NFKB1+non-targeting+JUN+MAP3K14,1
MAP3K14+RELA+FOXK1+CEBPD,1
IKBKG+GRK2+FOXC2+NFKB1,1


In [15]:
adata.obs['condition'].value_counts()

,count
condition,
targeting,93480
non-targeting,1482
